In [ ]:
from dask_jobqueue import LSFCluster
from dask.distributed import Client
import dask.dataframe as dd
import dask.array as da

In [ ]:
cluster = LSFCluster(cores=9, memory='25GB')
cluster

In [ ]:
client = Client(cluster)
client

In [ ]:
df = dd.demo.make_timeseries(start='2000-01-01',
                             end='2010-12-31',
                             dtypes={'x': float, 'y': float, 'id': int},
                             freq='10ms',
                             partition_freq='24h')
df

In [ ]:
df.head()

In [ ]:
df = df.persist()

In [ ]:
# Calculate the mean of the x and y columns for different ids
%time df.groupby('id')[['x', 'y']].mean().compute()

In [ ]:
# Calculate a rolling 1 minute window. Take the standard deviation for each windows. 
# Remove the first and last day of data. Find the index that corresponds to the maximum.
%time df.x.rolling('1min').std().loc['2000-01-02':'2010-12-30'].idmax().compute()

In [ ]:
# Calculate the singular vector decomposition
%time u, s, v = da.linalg.svd(df.values + 1)